In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

In [5]:
df_completo=pd.read_csv(f"Datasets/Data_ETL_EDA.csv")

In [6]:
def recomendation(title: str):
    """Ingresas un nombre de pelicula y te recomienda las similares en una lista"""
    title = title.lower()
    # Buscamos la película en el df de películas
    movie_id = df_completo.loc[df_completo["title"].str.lower() == title].index

    # Si la película no existe en el DataFrame, se devuelve nulo
    if movie_id.empty:
        return {"lista recomendada": "No se encontró la película"}

    # Intentamos obtener la sinopsis de la película ingresada por el usuario
    try:
        movie_overview = df_completo.loc[movie_id, 'overview'].iloc[0]
    except KeyError:
        return {"lista recomendada": "No hay suficiente información de esta película para hacer alguna recomendación"}

    # Creamos un vectorizador TF-IDF para convertir las sinopsis en una matriz de características
    vectorizer = TfidfVectorizer()
    matriz_caracteristicas = vectorizer.fit_transform(df_completo['overview'])

    # Calculamos la similitud de coseno entre la sinopsis de la película ingresada por el usuario y todas las demás sinopsis
    cosine_sim = cosine_similarity(vectorizer.transform([movie_overview]), matriz_caracteristicas)

    # Obtenemos los 5 índices de las películas más similares excluyendo la primera que es la que ingresó el usuario
    similar_movies = cosine_sim.argsort()[0][::-1][1:6]

    recomendation_list = []
    # Obtenemos los títulos de las películas similares y los añadimos a la lista de recomendación
    for i in similar_movies:
        recomendation_list.append((df_completo["title"].iloc[i]).title())

    return {"lista recomendada": sorted(recomendation_list)}


In [7]:
recomendation("batman")

{'lista recomendada': ['Batman Beyond: Return Of The Joker',
  'Batman Unlimited: Monster Mayhem',
  'Batman Vs Dracula',
  'Batman: Under The Red Hood',
  'The Dark Knight Rises']}

In [8]:
recomendation("toy story")

{'lista recomendada': ["Andy Hardy'S Blonde Trouble",
  'Small Fry',
  'The 40 Year Old Virgin',
  'Toy Story 2',
  'Toy Story 3']}